# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd9300d9760>
├── 'a' --> tensor([[ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013]])
└── 'x' --> <FastTreeValue 0x7fd9300d9b50>
    └── 'c' --> tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                        [ 1.9513,  0.0644,  1.0288, -1.8989],
                        [ 0.3587, -1.0831, -0.7567, -0.3360]])

In [4]:
t.a

tensor([[ 0.5375,  0.1991,  0.7136],
        [ 1.0259, -0.3141,  1.7013]])

In [5]:
%timeit t.a

78 ns ± 0.141 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd9300d9760>
├── 'a' --> tensor([[ 8.7054e-01,  3.7311e-01, -1.9478e+00],
│                   [-2.2847e-02, -1.9015e-03, -5.5027e-01]])
└── 'x' --> <FastTreeValue 0x7fd9300d9b50>
    └── 'c' --> tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                        [ 1.9513,  0.0644,  1.0288, -1.8989],
                        [ 0.3587, -1.0831, -0.7567, -0.3360]])

In [7]:
%timeit t.a = new_value

81.6 ns ± 0.436 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013]]),
    x: Batch(
           c: tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                      [ 1.9513,  0.0644,  1.0288, -1.8989],
                      [ 0.3587, -1.0831, -0.7567, -0.3360]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5375,  0.1991,  0.7136],
        [ 1.0259, -0.3141,  1.7013]])

In [11]:
%timeit b.a

77 ns ± 0.128 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2737,  0.3487, -1.2324],
               [-1.2704, -0.8477,  0.8558]]),
    x: Batch(
           c: tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                      [ 1.9513,  0.0644,  1.0288, -1.8989],
                      [ 0.3587, -1.0831, -0.7567, -0.3360]]),
       ),
)

In [13]:
%timeit b.a = new_value

599 ns ± 0.285 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

965 ns ± 3.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 67.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

165 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

164 µs ± 742 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd9300d31f0>
├── 'a' --> tensor([[[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]],
│           
│                   [[ 0.5375,  0.1991,  0.7136],
│                    [ 1.0259, -0.3141,  1.7013]]])
└── 'x' --> <FastTreeValue 0x7fd884da7f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.3 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd884d8a670>
├── 'a' --> tensor([[ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013],
│                   [ 0.5375,  0.1991,  0.7136],
│                   [ 1.0259, -0.3141,  1.7013]])
└── 'x' --> <FastTreeValue 0x7fd884d8a400>
    └── 'c' --> tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                        [ 1.9513,  0.0644,  1.0288, -1.8989],
                 

In [23]:
%timeit t_cat(trees)

35 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

63.1 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]],
       
               [[ 0.5375,  0.1991,  0.7136],
                [ 1.0259, -0.3141,  1.7013]]]),
    x: Batch(
           c: tensor([[[-0.1177,  0.2996, -0.3899, -0.0568],
                       [ 1.9513,  0.0644,  1.0288, -1.8989],
                       [ 0.3587, -1.0831, -0.7567, -0.3360]],
         

In [26]:
%timeit Batch.stack(batches)

90.9 µs ± 251 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013],
               [ 0.5375,  0.1991,  0.7136],
               [ 1.0259, -0.3141,  1.7013]]),
    x: Batch(
           c: tensor([[-0.1177,  0.2996, -0.3899, -0.0568],
                      [ 1.9513,  0.0644,  1.0288, -1.8989],
                      [ 0.3587, -1.0831, -0.7567, -0.3360],
                      [-0.1177,  0.2996, -0.3899, -0.0568],
                      [ 1.9513,  

In [28]:
%timeit Batch.cat(batches)

168 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

385 µs ± 935 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
